In [ ]:
import cv2
import numpy as np
import math
face_cascade = cv2.CascadeClassifier('C:/Users/Bernard/Anaconda3/pkgs/opencv-3.4.1-py35_200/Library/etc/haarcascades/haarcascade_frontalface_default.xml')
thickness = 3
iron_man = cv2.imread('Ironman.png',-1)
spider_man = cv2.imread('Spiderman.png',-1)
hulk = cv2.imread("Hulk.png",-1)
black_panther = cv2.imread("Black_panther.png",-1)
cap = cv2.VideoCapture(0)
def put_iron_mask(iron_man, img, x, y, w, h):
    iron_mask = iron_man[:,:,3]
    iron_mask_inv = cv2.bitwise_not(iron_mask)
    iron_man = iron_man[:,:,0:3]
    for(x,y,w,h) in faces:
        iron_man = cv2.resize(iron_man, (w,h),
                              interpolation = cv2.INTER_AREA)
        iron_mask = cv2.resize(iron_mask, (w,h),
                               interpolation = cv2.INTER_AREA)
        iron_mask_inv = cv2.resize(iron_mask_inv, (w,h),
                                   interpolation = cv2.INTER_AREA)
        frame_roi = img[y:y+h,x:x+w]
        masked_frame = cv2.bitwise_and(frame_roi, frame_roi, 
                                       mask=iron_mask_inv)
        masked_face = cv2.bitwise_and(iron_man, iron_man, mask=iron_mask) 
        img[y:y+h,x:x+w] = cv2.add(masked_frame, masked_face)  
    
    return img

def put_spider_mask(spider_man, img, x, y, w, h):
    spider_mask = spider_man[:,:,3]
    spider_mask_inv = cv2.bitwise_not(spider_mask)
    spider_man = spider_man[:,:,0:3]
    for(x,y,w,h) in faces:
        spider_man = cv2.resize(spider_man, (w,h),
                                interpolation = cv2.INTER_AREA)
        spider_mask = cv2.resize(spider_mask, (w,h),
                                 interpolation = cv2.INTER_AREA)
        spider_mask_inv = cv2.resize(spider_mask_inv, (w,h),
                                     interpolation = cv2.INTER_AREA)
        frame_roi = img[y:y+h,x:x+w]
        masked_frame = cv2.bitwise_and(frame_roi, frame_roi, 
                                       mask=spider_mask_inv)
        masked_face = cv2.bitwise_and(spider_man, spider_man, 
                                      mask=spider_mask) 
        img[y:y+h,x:x+w] = cv2.add(masked_frame, masked_face)  
    
    return img

def put_hulk_mask(hulk, img, x, y, w, h):
    hulk_mask = hulk[:,:,3]
    hulk_mask_inv = cv2.bitwise_not(hulk_mask)
    hulk = hulk[:,:,0:3]
    for(x,y,w,h) in faces:
        hulk = cv2.resize(hulk, (w,h),
                          interpolation = cv2.INTER_AREA)
        hulk_mask = cv2.resize(hulk_mask, (w,h),
                               interpolation = cv2.INTER_AREA)
        hulk_mask_inv = cv2.resize(hulk_mask_inv, (w,h),
                                   interpolation = cv2.INTER_AREA)
        frame_roi = img[y:y+h,x:x+w]
        masked_frame = cv2.bitwise_and(frame_roi, frame_roi, 
                                       mask=hulk_mask_inv)
        masked_face = cv2.bitwise_and(hulk, hulk, mask=hulk_mask) 
        img[y:y+h,x:x+w] = cv2.add(masked_frame, masked_face)  
    
    return img

def put_black_mask(black_panther, img, x, y, w, h):
    black_mask = black_panther[:,:,3]
    black_mask_inv = cv2.bitwise_not(black_mask)
    black_panther = black_panther[:,:,0:3]
    for(x,y,w,h) in faces:
        black_panther = cv2.resize(black_panther, (w,h),
                                   interpolation = cv2.INTER_AREA)
        black_mask = cv2.resize(black_mask, (w,h),
                                interpolation = cv2.INTER_AREA)
        black_mask_inv = cv2.resize(black_mask_inv, (w,h),
                                    interpolation = cv2.INTER_AREA)
        frame_roi = img[y:y+h,x:x+w]
        masked_frame = cv2.bitwise_and(frame_roi, frame_roi, 
                                       mask = black_mask_inv)
        masked_face = cv2.bitwise_and(black_panther, black_panther, 
                                      mask = black_mask) 
        img[y:y+h,x:x+w] = cv2.add(masked_frame, masked_face)  
    
    return img

while(cap.isOpened()):
    # read image
    ret, img = cap.read()
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    # get hand data from the rectangle sub window on the screen
    cv2.rectangle(img, (300,300), (100,100), (0,255,0),0)
    crop_img = img[100:300, 100:300]

    # convert to grayscale
    gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)

    # applying gaussian blur
    value = (35, 35)
    blurred = cv2.GaussianBlur(gray, value, 0)

    # thresholdin: Otsu's Binarization method
    _, thresh1 = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    image, contours, hierarchy = cv2.findContours(thresh1.copy(), 
                                                  cv2.RETR_TREE, 
                                                  cv2.CHAIN_APPROX_NONE)
    
    # find contour with max area
    cnt = max(contours, key = lambda x: cv2.contourArea(x))

    # create bounding rectangle around the contour 
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(crop_img, (x, y), (x+w, y+h), (0, 0, 255), 0)

    # finding convex hull
    hull = cv2.convexHull(cnt)

    # drawing contours
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing, [cnt], 0, (0, 255, 0), 0)
    cv2.drawContours(drawing, [hull], 0,(0, 0, 255), 0)

    # finding convex hull
    hull = cv2.convexHull(cnt, returnPoints=False)

    # finding convexity defects
    defects = cv2.convexityDefects(cnt, hull)
    count_defects = 0
    cv2.drawContours(thresh1, contours, -1, (0, 255, 0), 3)

    # applying Cosine Rule to find angle for all defects (between fingers)
    # with angle > 90 degrees and ignore defects
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]

        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])

        # find length of all sides of triangle
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)

        # apply cosine rule here
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57

        # ignore angles > 90 and highlight rest with red dots
        if angle <= 90:
            count_defects += 1
            cv2.circle(crop_img, far, 1, [0,0,255], -1)

        # draw a line from start to end i.e. the convex points (finger tips)
        
        cv2.line(crop_img,start, end, [0,255,0], 2)
        #cv2.circle(crop_img,far,5,[0,0,255],-1)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        if count_defects == 1:
            cv2.putText(img,"I am Iron Man", (0, 50), 
                        cv2.FONT_HERSHEY_TRIPLEX, 1,(255,0,0),1, cv2.LINE_AA)
            put_iron_mask(iron_man, img, x, y, w, h)
        elif count_defects == 2:
            cv2.putText(img, "I am Spider Man", (0, 50), 
                        cv2.FONT_HERSHEY_TRIPLEX, 1,(255,0,0),1, cv2.LINE_AA)
            put_spider_mask(spider_man, img, x, y, w, h)
        elif count_defects == 3:
            cv2.putText(img,"I am Hulk", (0, 50), 
                        cv2.FONT_HERSHEY_TRIPLEX, 1,(255,0,0),1, cv2.LINE_AA)
            put_hulk_mask(hulk, img, x, y, w, h)
        elif count_defects > 3:
            cv2.putText(img,"I am Black Panther", (0, 50), 
                        cv2.FONT_HERSHEY_TRIPLEX, 1,(255,0,0),1, cv2.LINE_AA)
            put_black_mask(black_panther, img, x, y, w, h)
        else:
            cv2.putText(img,"Hello World!!!", (0, 50),
                        cv2.FONT_HERSHEY_TRIPLEX, 1,(255,0,0),1, cv2.LINE_AA)
    
    cv2.imshow('Gesture', img)
    #all_img = np.hstack((drawing, crop_img))
    #cv2.imshow('Contours', all_img)
    k = cv2.waitKey(20) & 0xFF
    if k == 27:
        break

cv2.waitKey()    
cap.release()
cv2.destroyAllWindows()